## Modelo de Machine Learning


Se emplearán los siguientes métodos y técnicas de Procesamiento de Lenguaje Natural:

- **TF-Idata_ML (Term Frequency-Inverse Document Frequency)**: Esta técnica transforma el texto en una representación numérica (matriz de características), asignando a cada término un valor que indica su relevancia en un documento, en relación con el conjunto de textos (corpus). Los valores obtenidos reflejan únicamente la frecuencia y relevancia de las palabras en el contexto textual.

- **Similitud del Coseno**: Esta técnica evalúa la similitud entre dos vectores generados por TF-Idata_ML, midiendo el grado de semejanza en el espacio vectorial. Se basa únicamente en las palabras y sus ponderaciones, sin verse influida por valores numéricos externos.

## Cargue de datos luego de aplicado el EDA

In [130]:
#Importar librerías
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [131]:
data_ML = pd.read_parquet('../datasets/complete_data.parquet')

In [132]:
data_ML.dtypes

budget                    int64
id                        int64
original_language        object
overview                 object
popularity              float64
release_date             object
revenue                 float64
runtime                 float64
status                   object
tagline                  object
title                    object
vote_average            float64
vote_count              float64
collection_name          object
genres                   object
production_companies     object
production_countries     object
spoken_languages         object
return                  float64
release_year              int64
directors                object
actors                   object
dtype: object

In [133]:
data_ML['title'][:30]

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
5                               Heat
6                            Sabrina
7                       Tom and Huck
8                       Sudden Death
9                          GoldenEye
10            The American President
11       Dracula: Dead and Loving It
12                             Balto
13                             Nixon
14                  Cutthroat Island
15                            Casino
16             Sense and Sensibility
17                        Four Rooms
18    Ace Ventura: When Nature Calls
19                       Money Train
20                        Get Shorty
21                           Copycat
22                         Assassins
23                            Powder
24                 Leaving Las Vegas
25                           Othello
26                      Now and Then
2

In [134]:
data_ML.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection_name', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'return', 'release_year', 'directors', 'actors'],
      dtype='object')

In [135]:
#revisar valores nulos
data_ML.isnull().sum()

budget                      0
id                          0
original_language           9
overview                  903
popularity                  0
release_date                0
revenue                     0
runtime                   229
status                     76
tagline                 24240
title                       0
vote_average                0
vote_count                  0
collection_name         39998
genres                   2289
production_companies    11478
production_countries     6027
spoken_languages         3764
return                      0
release_year                0
directors                   0
actors                      0
dtype: int64

In [136]:
data_ML.shape

(44425, 22)

**Se crearán dos columnas como resultado del análisis efectuado en el EDA**

In [137]:
data_ML['collection_name'][:3]

0         Toy Story Collection
1                         None
2    Grumpy Old Men Collection
Name: collection_name, dtype: object

In [138]:
# Extraer el primer actor de la lista en la columna 'actors'
data_ML['first_actor'] = data_ML['actors'].apply(lambda x: x.split(',')[0] if pd.notna(x) else '')

# Extraer el primer director de la lista en la columna 'director'
data_ML['first_director'] = data_ML['directors'].apply(lambda x: x.split(',')[0] if pd.notna(x) else '')

In [139]:
data_ML.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genres,production_companies,production_countries,spoken_languages,return,release_year,directors,actors,first_actor,first_director
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,None,...,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English,12.451801,1995,John Lasseter,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...",Tom Hanks,John Lasseter
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français",4.043035,1995,Joe Johnston,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",Robin Williams,Joe Johnston
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English,0.000000,1995,Howard Deutch,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...",Walter Matthau,Howard Deutch
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English,5.090760,1995,Forest Whitaker,"Whitney Houston, Angela Bassett, Loretta Devin...",Whitney Houston,Forest Whitaker
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English,0.000000,1995,Charles Shyer,"Steve Martin, Diane Keaton, Martin Short, Kimb...",Steve Martin,Charles Shyer


In [140]:
data_ML.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'collection_name', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'return', 'release_year', 'directors', 'actors', 'first_actor',
       'first_director'],
      dtype='object')

Se utilizará el atributo overview en combinación de otros más para evaluar los distintos modelos

In [141]:
data_ML = data_ML.fillna('')  # Replace None with empty strings

## **Model1**

In [142]:
#Extrar las columnas relevantes para el modelo
model1 = data_ML[['title', 'overview']].copy()

In [143]:
# Crear una instancia de Tfidata_MLVectorizer
tfidata_ML_1 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

In [144]:
# Aplicar la transformación TF-Idata_ML al contenido
tfidata_ML_matriz_1 = tfidata_ML_1.fit_transform(model1['overview'])


In [145]:
# Función para obtener recomendaciones
def recomendacion_m1(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model1.index, index=model1['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model1.duplicated(['title']).any():
            primer_ind = model1[model1['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidata_ML_matriz_1[ind], tfidata_ML_matriz_1).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model1)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model1.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

## **Se realizan distintas pruebas para evaluar el modelo**

In [146]:

recomendacion_m1('The Dark Knight Rises')

['Batman Forever',
 'The Dark Knight',
 'Batman',
 'Batman Returns',
 'Batman: Under the Red Hood']

In [147]:

recomendacion_m1('Jumanji')

['Word Wars', 'The Bar', 'Table No. 21', 'Quintet', 'The Dark Angel']

In [148]:

recomendacion_m1('Titanic')

['Grantham and Rose',
 'The Legend of 1900',
 'Raise the Titanic',
 'Dustbin Baby',
 'The Experience']

In [149]:

recomendacion_m1('Wonder Woman')

['Wide Eyed and Legless',
 'Trevor Noah: The Daywalker',
 'Nightbeast',
 'The Machinist',
 'Escape from New York']

In [150]:

recomendacion_m1('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Samson and the Seven Miracles of the World']

In [151]:

recomendacion_m1('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'Prince of Persia: The Sands of Time',
 'Doctor in Distress']

In [152]:
# Se extraen las columnas necesarias para el Sistema de Recomendación
model1 = data_ML[['title', 'overview']]
model1.to_parquet('../datasets/movies_model1.parquet')

## **Model2**

In [153]:
#Extrar las columnas relevantes para el modelo
model2 = data_ML[['title', 'overview', 'first_director']].copy()

In [154]:
# Crear una instancia de Tfidata_MLVectorizer
tfidata_ML_2 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))

In [155]:
model2['concatenado'] = model2['overview'] + ' ' + model2['first_director']

In [156]:
# Aplicar la transformación TF-Idata_ML
tfidata_ML_matriz_2 = tfidata_ML_2.fit_transform(model2['concatenado'])

In [157]:
# Función para obtener recomendaciones
def recomendacion_m2(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model2.index, index=model2['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model2.duplicated(['title']).any():
            primer_ind = model2[model1['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidata_ML_matriz_2[ind], tfidata_ML_matriz_2).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model2)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model2.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones


In [158]:
recomendacion_m2('Titanic')

['Grantham and Rose',
 'The Legend of 1900',
 'Dustbin Baby',
 'Women of the Prehistoric Planet',
 'Raise the Titanic']

In [159]:
recomendacion_m2('Avatar')

['The Robbery of the Third Reich',
 'Bloodbrothers',
 'Stand by Me Doraemon',
 'The War of the Robots',
 'Bill Hicks: Revelations']

In [160]:
recomendacion_m2('Wonder Woman')

['Wide Eyed and Legless',
 'The Machinist',
 'Escape from New York',
 'Diana',
 'Our Dancing Daughters']

In [161]:
recomendacion_m2('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 'Ale e Franz - È tanto che aspetti?']

In [162]:
recomendacion_m2('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'Prince of Persia: The Sands of Time',
 'Doctor in Distress']

In [163]:
recomendacion_m2('Shrek')

['Shrek 2',
 'Shrek the Third',
 'We Stand Alone Together',
 'Silk Stockings',
 'Scared Shrekless']

## **Model3**

In [164]:
#Extrar las columnas relevantes para el modelo
model3 = data_ML[['title', 'overview', 'genres', 'first_director']].copy()

In [165]:
##Se separan los géneros y se convierten en palabras individuales
model3['genres'] = model3['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [166]:
# Se crea una instancia de la clase Tfidata_MLVectorizer
tfidata_ML_3 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-Idata_ML al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidata_ML_matriz_3 = tfidata_ML_3.fit_transform(model3['overview'] + ' ' + model3['genres'] + ' ' + model3['first_director'])

In [167]:
tfidata_ML_matriz_3.shape

(44425, 1093045)

In [168]:
# Función para obtener recomendaciones
def recomendacion_m3(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model3.index, index=model3['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model1.duplicated(['title']).any():
            primer_ind = model3[model3['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidata_ML_matriz_3[ind], tfidata_ML_matriz_3).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model3)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model3.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [169]:
recomendacion_m3('Titanic')

['Grantham and Rose',
 'Drona',
 'The Legend of 1900',
 'Dustbin Baby',
 'Raise the Titanic']

In [170]:
recomendacion_m3('Avatar')

['The War of the Robots',
 'The Robbery of the Third Reich',
 'Avatar 2',
 'Stand by Me Doraemon',
 'Bloodbrothers']

In [171]:
recomendacion_m3('Wonder Woman')

['Wide Eyed and Legless',
 'Escape from New York',
 'Dinotopia: Quest for the Ruby Sunstone',
 'Nightbeast',
 'The Machinist']

In [172]:
recomendacion_m3('Star Wars')

['The Empire Strikes Back',
 'The Star Wars Holiday Special',
 'Star Wars: The Force Awakens',
 'Return of the Jedi',
 "Bill & Ted's Bogus Journey"]

In [173]:
recomendacion_m2('The Avengers')

['The Work and the Glory',
 'Monty Python and the Holy Grail',
 "Sir Arne's Treasure",
 'Prince of Persia: The Sands of Time',
 'Doctor in Distress']

**Se decide probar con los atributos 'title', 'collection_name', 'first_actor', 'first_director' esperando tener una posible mejora en cuanto a la precisión de las recomendaciones**

## **Model4**

In [174]:
#Extrar las columnas relevantes para el modelo
model4 = data_ML[['title', 'genres', 'first_actor', 'first_director']].copy()

In [175]:
#Se separan los géneros y se convierten en palabras individuales
model4['genres'] = model4['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))

In [176]:
#Se crea una instancia de la clase Tfidata_MLVectorizer
tfidata_ML_4 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
# Aplicar la transformación TF-Idata_ML al texto contenido en las columnas "overview_clean", "genres" y "director" del dataframe 'modelo4'
tfidata_ML_matriz_4 = tfidata_ML_4.fit_transform(model4['genres'] + ' ' + model4['first_actor'] + ' ' + model4['first_director'])

In [177]:
# Función para obtener recomendaciones
def recomendacion_m4(titulo):
    #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model4.index, index=model4['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model4.duplicated(['title']).any():
            primer_ind = model4[model4['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidata_ML_matriz_4[ind], tfidata_ML_matriz_4).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model4)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model4.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [178]:
recomendacion_m4('Titanic')

['The Deaths of Ian Stone', 'GoldenEye', 'Godsend', 'Pusher', 'Thirst']

In [179]:
recomendacion_m4('Avatar')

["Being Elmo: A Puppeteer's Journey",
 'The H-Man',
 'Summer in Tyrol',
 'Tracks',
 'The River Wild']

In [180]:
recomendacion_m4('Shrek')

['Office Killer',
 'Little Secrets',
 'A Monster in Paris',
 'Western Union',
 'The Bullfighters']

In [181]:
recomendacion_m4('Star Wars')

['The Empire Strikes Back',
 'Return of the Jedi',
 "Bill & Ted's Bogus Journey",
 'Eddie Izzard: Dress to Kill',
 'Xtro 2: The Second Encounter']

In [182]:
recomendacion_m4('Toy Story')

['Funny Farm',
 'And the Band Played On',
 'National Treasure: Book of Secrets',
 'Troop Beverly Hills',
 'Over the Hedge']

In [183]:
recomendacion_m4('Star Trek')

['Unbreakable',
 'The Treasure of the Sierra Madre',
 "Girls Can't Swim",
 'Apocalypse Now',
 'Mayor of the Sunset Strip']

In [184]:
recomendacion_m4('Mars')

["Encounter at Raven's Gate",
 'Blazing a Trail to the Stars',
 'Passengers',
 'White Out, Black In',
 'Beloved']

In [185]:
recomendacion_m4('Avatar')

["Being Elmo: A Puppeteer's Journey",
 'The H-Man',
 'Summer in Tyrol',
 'Tracks',
 'The River Wild']

In [186]:
recomendacion_m4('The Hunger Games')

['Piranhaconda',
 'Love, etc.',
 'Weekend of a Champion',
 'Rosewater',
 'Eddie Murphy Raw']

In [187]:
recomendacion_m4('Forrest Gump')

['Heartburn',
 'Sleepless in Seattle',
 'Arthur',
 'A Summer in St. Tropez',
 'Jersey Girl']

In [188]:
recomendacion_m4('Men in Black')

['Cyborg',
 'Godzilla vs. Biollante',
 'Phase IV',
 'Beginning of the End',
 'Dirty Mary Crazy Larry']

In [189]:
recomendacion_m4('Wonder Woman')

['There Will Come Soft Rains',
 'Elephants Dream',
 'More',
 'Invaders from the Deep',
 'The Call Up']

## **Model5**

In [190]:
#Extrar las columnas relevantes para el modelo
model5 = data_ML[['title', 'genres', 'tagline' ,'first_actor', 'first_director']].copy()

In [191]:
#Se separan los géneros y se convierten en palabras individuales
model5['genres'] = model5['genres'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
#Se separan los slogans y se convierten en palabras individuales
model5['tagline'] = model5['tagline'].fillna('').apply(lambda x: ' '.join(x.replace(',', ' ').replace('-', '').lower().split()))
#Se crea una instancia de la clase Tfidata_MLVectorizer
tfidata_ML_5 = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
#Aplicar la transformación TF-Idata_ML y obtener matriz numérica
tfidata_ML_matriz_5 = tfidata_ML_5.fit_transform(model5['genres'] + ' ' + model5['tagline'] + ' ' + model5['first_actor']+ ' ' + model5['first_director'])

In [192]:
# Función para obtener recomendaciones
def recomendacion_m5(titulo):
  #Crear una serie que asigna un índice a cada título de las películas
    indices = pd.Series(model5.index, index=model5['title']).drop_duplicates()
    if titulo not in indices:
        return 'La película ingresada no se encuentra en la base de datos'
    else:
        #Obtener el índice de la película que coincide con el título
        ind = pd.Series(indices[titulo]) if titulo in indices else None
        #Si el título de la película está duplicado, devolver el índice de la primera aparición del título en el DataFrame
        if model5.duplicated(['title']).any():
            primer_ind = model5[model5['title'] == titulo].index[0]
            if not ind.equals(pd.Series(primer_ind)):
                ind = pd.Series(primer_ind)
        #Calcular la similitud coseno entre la película de entrada y todas las demás películas en la matriz de características
        cosine_sim = cosine_similarity(tfidata_ML_matriz_5[ind], tfidata_ML_matriz_5).flatten()
        simil = sorted(enumerate(cosine_sim), key=lambda x: x[1], reverse=True)[1:6]
        #Verificar que los índices obtenidos son válidos
        valid_ind = [i[0] for i in simil if i[0] < len(model5)]
        #Obtener los títulos de las películas más similares utilizando el índice de cada película
        recomendaciones = model5.iloc[valid_ind]['title'].tolist()
        #Devolver la lista de títulos de las películas recomendadas
        return recomendaciones

In [193]:
recomendacion_m5('Toy Story')

['And the Band Played On',
 'National Treasure: Book of Secrets',
 'The Fourth World War',
 '47 Samurai',
 'Late Bloomers']

In [194]:
recomendacion_m5('Wonder Woman')

['Macross Plus: Movie Edition',
 'DC Showcase: Catwoman',
 'Mobile Suit Gundam II: Soldiers of Sorrow',
 'There Will Come Soft Rains',
 'Elephants Dream']

In [195]:
recomendacion_m5('Avatar')

["Being Elmo: A Puppeteer's Journey",
 'Summer in Tyrol',
 'Blackbird',
 'Bleak Moments',
 'Tracks']

In [196]:
recomendacion_m5('Batman Returns')

['The Lords of Flatbush',
 'Time to Leave',
 'Fresh',
 'Les Dames du Bois de Boulogne',
 'Casual Sex?']

In [197]:
recomendacion_m5('Star Wars')

['The Star Wars Holiday Special',
 'The Galaxy Invader',
 "On Her Majesty's Secret Service",
 'The Empire Strikes Back',
 'The Pied Piper']

In [198]:
recomendacion_m5('The Avengers')

['Medusa',
 'Get Smart',
 'Bigfoot: The Lost Coast Tapes',
 'The Man from U.N.C.L.E.',
 'Super Fuzz']

In [199]:
recomendacion_m5('The Dark Knight Rises')

['The Memory Of A Killer',
 'Trapped in the Closet: Chapters 13-22',
 'The Little Girl Who Conquered Time',
 'Civil Brand',
 'Elizabethtown']

In [200]:
recomendacion_m5('Jumanji')

['Gunner Palace',
 'When Evening Falls on Bucharest or Metabolism',
 'The Living Room of the Nation',
 'An Innocent Man',
 'My Name Is Joe']

In [201]:
recomendacion_m5('Titanic')

['Lenny',
 'Restless Blood',
 'Jonny Quest Vs. The Cyber Insects',
 'Robotrix',
 'Bhutto']

In [202]:
recomendacion_m5('Men in Black')

['All the Rage',
 'Bomb It',
 'Spirit of the Marathon',
 'King and the Clown',
 'Scooby-Doo! Legend of the Phantosaur']

## **Modelo seleccionado**
El modelo Model3 fue el que presentó una mejor recomendación pero tiene un mayor costo computacional. Es por ello que elegimos el segundo mejor modelo, Model5, que podrá adaptarse a las limitaciones de memoria del sitio render.

In [203]:
# Se extraen las columnas necesarias para el Sistema de Recomendación
model5 = data_ML[['title', 'genres', 'tagline' ,'first_actor', 'first_director']]

In [204]:
# Se exportan los datos a formato parquet
model5.to_parquet('movies_model5.parquet')

In [205]:
model5.shape

(44425, 5)